<a href="https://colab.research.google.com/github/saas-for-dgv/AI/blob/main/A_BigramModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Use this to enable word-wrap in outputs. This is only to be used for google colab notebook. If you are running
# this in any other environment, omit this entire code.
from IPython.display import HTML, display

def set_css():
  display(HTML("""
  <style>
    pre { white-space: pre-wrap; }
  </style>
  """))

get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tensorflow as tf
import numpy as np
import time
import gc
import os

os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

In [ ]:
with open('./wizard_of_oz.txt', 'r') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print("vocab size =",len(chars))

In [ ]:
# Hyper parameters for training
block_size = 8
batch_size = 4
vocab_size = len(chars)
max_iters = 1000
eval_iters = 500#250
learning_rate = 3e-4 #
#dropout = 0.2 # You might want to disable this in eval mode!!

# prompt: detect gpu device for different os using torch. dont use nested if else. write a better code
device = 'mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
# helpers
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

# x,y = get_batch('train')
# print(x, 'targets', y)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding (vocab_size,vocab_size)

    # Best practice to write forward pass from scratch
    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index)
        loss = None
        if targets is None:
            return logits, loss
        # B = Batch, T = Time, Channels
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)

            # focus only on the last time step
            logits = logits[ :, -1, :] # becomes (B, C)

            # apply softmax to get probabilites
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device=device)

In [ ]:
# create a PyTorch optimizer. AdamW is Adam Weight decay. This is a training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter%eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss:{losses['val']:.3f}")
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
#print(loss.item())

In [ ]:
# Generate based on above loss
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model.to(device=device).generate(context, max_new_tokens=500)[0].tolist())
generated_chars